## Initial Setup

In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1663453062035_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#spark.sparkContext.getConf().get('spark.driver.memory')


### Run this cell only to increase memory on spark memory

In [ ]:
#%%configure -f 
#{"driverMemory": "6000M"}

In [ ]:
#spark.sparkContext.getConf().get('spark.driver.memory')


In [4]:
# Run this everytime you create a new spark instance. 

spark.sparkContext.install_pypi_package("plotly==5.5.0")
spark.sparkContext.install_pypi_package("pandas==0.25.1")
spark.sparkContext.install_pypi_package("numpy==1.14.5")
spark.sparkContext.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b3/f4/23d4a698db9fe772f7fdf40ac17b743c4b0d80274732c59db5bd45acb3be/plotly-5.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/a5/f86bc8d67c979020438c8559cc70cfe3a1643fd160d35e09c9cca6a09189/tenacity-8.0.1-py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/3f/e7/7f24ef402a5766c677683e313c5595137d754cb9eb1c99627803280e79d5/numpy-1.14.5-cp37-cp37m-manylinux1_x86_64.whl
  Found existing installation: numpy 1.16.5
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1663458587571-0

  Using cached https://files

In [5]:
# Write all import commands here 
from pyspark.sql import functions as F
from pyspark.sql.functions import mean as _mean, stddev as _stddev, stddev as _count, col
from pyspark.ml.feature import Bucketizer
import matplotlib.pyplot as plt
import pandas as pd




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
#%matplot inline
x = list(range(1,10))
y = list(range(1,10))
plt.plot(x,y)

In [ ]:
#%matplot plt

In [6]:
def get_quantiles(df, col_name, quantiles_list=[0.01, 0.25, 0.5, 0.75, 0.99]):
    """
    Takes a numerical column and returns column values at requested quantiles

    Inputs 
    Argument 1: Dataframe
    Argument 2: Name of the column
    Argument 3: A list of quantiles you want to find. Default value [0.01, 0.25, 0.5, 0.75, 0.99]

    Output 
    Returns a dictionary with quantiles as keys and column quantile values as values 
    """
    # Get min, max and quantile values for given column
    zero = df.approxQuantile(col_name,[0.0] , 0.2)[0]
    quant_vals = df.approxQuantile(col_name, quantiles_list, 0.2)
    one = df.approxQuantile(col_name,[1.0] , 0.2)[0]

    # Store min, quantiles and max in output dict, sequentially
    quantiles_dict = {0.0:int(zero)}
    for i in range(0,len(quantiles_list)):
        quantiles_dict[quantiles_list[i]]=quant_vals[i]
    quantiles_dict[1.0]=int(one)

    return(quantiles_dict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Defining Custom Functions 

In [7]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

Test_data = [[25, 23, 21, 30, 11, 6, 1, 3, 5, 28, 4, 24, 24, 14, 25, 29, 15, 2, 27, 9, 11, 22, 11, 11, 11, 14, 1, 25, 8, 18, 20, 26, 13, 1, 4, 23, 20, 17, 9, 27, 4, 23, 26, 14, 13, 1, 27, 18, 19, 19],
             [21, 21, 16, 17, 8, 26, 16, 11, 9, 30, 25, 24, 5, 12, 26, 17, 11, 15, 5, 12, 3, 1, 10, 9, 15, 2, 5, 17, 4, 3, 12, 20, 21, 23, 23, 21, 26, 21, 14, 13, 16, 29, 26, 6, 14, 25, 14, 5, 18, 2],
             [18, 24, 26, 10, 28, 12, 1, 11, 30, 10, 1, 24, 5, 1, 3, 29, 2, 27, 5, 15, 23, 26, 14, 13, 6, 9, 28, 2, 27, 9, 26, 1, 28, 10, 27, 23, 12, 25, 28, 25, 8, 15, 11, 10, 11, 5, 14, 4, 3, 3],
             [15, 4, 15, 8, 1, 1, 28, 13, 26, 5, 27, 15, 11, 13, 29, 1, 5, 15, 12, 29, 9, 17, 20, 19, 7, 15, 26, 28, 5, 7, 8, 21, 14, 20, 15, 4, 5, 18, 6, 29, 12, 20, 18, 7, 18, 25, 27, 19, 6, 21]]

columns = ["Test_col_1", "Test_col_2", "Test_col_3", "Test_col_4"]

quantiles_test_df = spark.createDataFrame(Test_data, columns)

assert get_quantiles(quantiles_test_df, "Test_col_1", quantiles_list = [0.5]) == {0.0: 15, 0.5: 18.0, 1.0: 25}, "Recheck the get_quantiles function"
assert get_quantiles(quantiles_test_df, "Test_col_2", quantiles_list = [0.6]) == {0.0: 4, 0.6: 23.0, 1.0: 24}, "Recheck the get_quantiles function"
assert get_quantiles(quantiles_test_df, "Test_col_3", quantiles_list = [0.25]) == {0.0: 15, 0.25: 15.0, 1.0: 26}, "Recheck the get_quantiles function"
assert get_quantiles(quantiles_test_df, "Test_col_4") == {0.0: 8, 0.01: 8.0, 0.25: 8.0, 0.5: 10.0, 0.75: 17.0, 0.99: 30.0, 1.0: 30}, "Recheck the get_quantiles function"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:

def plot_bucketwise_statistics (summary, bucketizer):
    """
    Takes in a dataframe and a bucketizer object and plots the summary statistics for each bucket in the dataframe. 
  
    Inputs
    Argument 1: Pandas dataframe obtained from bucket_col_print_summary function 
    Argument 2: Bucketizer object obtained from bucket_col_print_summary function
  
    Output
    Displays a plot of bucketwise average ratings nunber of ratings of a parameter.   
    """
    # Creating bucket labels from splits
    classlist = bucketizer.getSplits()
    number_of_classes = len(classlist) - 1

    class_labels = []
    hover_labels = []
    for i in range (number_of_classes):
        hover_labels.append(str(classlist[i])+"-"+str(classlist[i+1]) +" (Bucket name: "+ str(int(i)) +")"  )
        class_labels.append(str(classlist[i])+"-"+str(classlist[i+1]) )
  
    summary["Scaled_number"] = (summary["n_ratings"]-summary["n_ratings"].min())/(summary["n_ratings"].max()-summary["n_ratings"].min()) + 1.5
    summary['Bucket_Names'] = class_labels
  
    # making plot
    x = summary["Bucket_Names"]
    y1 = summary["avg_rating"]
    y2 = summary["n_ratings"]
    err = summary["stddev_rating"]  

    # Plot scatter here
    plt.rcParams["figure.figsize"] = [summary.shape[0]+2, 6.0]
    plt.rcParams["figure.autolayout"] = True
    fig, ax1 = plt.subplots()

    bar = ax1.bar(x, y1, color = "#262261")
    ax1.errorbar(x, y1, yerr=err, fmt="o", color="#EE4036")
    ax1.set(ylim=(0, 7))
  
    #ax1.bar_label(bar , fmt='%.2f', label_type='edge')  
    def barlabel(x_list,y_list):
        for i in range(len(x_list)):
            ax1.text(i,y_list[i] + 0.2,y_list[i], ha = 'center',
  			         fontdict=dict(size=10),
  			         bbox=dict(facecolor='#262261', alpha=0.2)         
  			        )
    barlabel(summary["Bucket_Names"].tolist() ,summary["avg_rating"].round(2).tolist())
  
    ax2 = ax1.twinx()
    ax2.scatter(x, y2, s=summary["Scaled_number"]*500, c = '#FAAF40')  
    ax2.set(ylim=(0, summary["n_ratings"].max()*1.15))
    def scatterlabel(x_list,y_list):
  	    for i in range(len(x_list)):
  		    ax2.text(i,y_list[i] + 15000,y_list[i], ha = 'center',
  					 fontdict=dict(size=10),
                     bbox=dict(facecolor='#FAAF40', alpha=0.5)
  					)
    scatterlabel(summary["Bucket_Names"].tolist() ,summary["n_ratings"].tolist())
  
    # giving labels to the axises
    ax1.set_xlabel(bucketizer.getOutputCol(), fontdict=dict(size=14)) 
    ax1.set_ylabel("Average Ratings",fontdict=dict(size=14))
  
    # secondary y-axis label
    ax2.set_ylabel('Number of Ratings',fontdict=dict(size=14))
  
    #plot Title
    plt.title('Bucketwise average ratings and number of ratings for \n'+bucketizer.getInputCol(), 
              fontdict=dict(size=14))  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def bucket_col_print_summary(df, splits, inputCol, outputCol):
    """
    Given a numerical column in a data frame, adds a bucketized version of the column to the data frame, according to splits provided.
    Also prints a summary of ratings seen in each bucket made.

    Inputs 
    Argument 1: Data Frame 
    Argument 2: Values at which the column will be split
    Argument 3: Name of the input column (numerical column)
    Argument 4: Name of the output column (bucketized numerical column)

    Output: 
    1) New dataframe with the output column added
    2) Bucketizer object trained from the input column 
    3) Pandas dataframe with summary statistics for ratings seen in buckets of the output column
    Also plots summary statistics for ratings seen in buckets of the output column
    """

    # Dropping bucket if it already exists

    # Training bucketizer
    # bucketizer is an inbuilt feature transformer in SparkML 
    # use the bucketizer to create buckets and transfrom features
    bucket = Bucketizer(splits=splits, inputCol=inputCol, outputCol=outputCol)
    bucketizer = bucket.transform(df)
    
    # Printing meta information on buckets created
    print("Added bucketized column {}".format(outputCol))
    print("")
    print("Bucketing done for split definition: {}".format(splits))
    print("")  
    print("Printing summary statistics for ratings in buckets below:")
    
   
    # Creating a summary statistics dataframe and passing it to the plotting function
    # Write code to create a summary data frame. Ensure the summary df is pandas df. 
    # The data frame must have average ratings per bucket, standard deviation per bucket, number of ratings per bucket
    summary_df = bucketizer.groupBy(outputCol).agg(
        F.avg("rating").alias("avg_rating"),
        F.stddev("rating").alias("stddev_rating"),
        F.count("rating").alias("n_ratings"),
        )
    summary = summary_df.withColumnRenamed(outputCol,"Bucket_Names").toPandas()
    
    # uncomment the line below after you ensure this function is wokring correctly.  
    plot_bucketwise_statistics(summary,bucket)
  
    return df, bucketizer, summary

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
def get_column_distribution_summary(df, col_name):
    """
    Takes a column in a data frame and prints the summary statistics (average, standard deviation, count and distinct count) 
    for all unique values in that column.
  
    Inputs 
    Argument 1: Dataframe 
    Argument 2: Name of the column
  
    Output
    Returns nothing 
    Prints a Dataframe with summary statistics
    """
    
    print(
          # add code here 
          df.groupBy("n_steps").agg(
              F.avg("rating").alias("avg_rating"),
              F.stddev("rating").alias("stddev_rating"),
              F.count("rating").alias("n_ratings"),
              F.countDistinct("recipe_id").alias("n_recipes")
              ).sort("n_steps").show()
          )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def get_n_items_satisfying_condition (df, condition, aggregation_level = "recipe"):
    """
    Given a condition, find the number of recipes / reviews that match the condition.
    Also calculates the percentage of such recipes / reviews as a percentage of all recipes / reviews.
  
    Inputs 
    Argument 1: Dataframe 
    Argument 2: Logical expression describing a condition, string type. eg: "minutes == 0"
    Argument 3: Aggregation level for determining "items", either  "recipe" or "review". Default value == "recipe"
  
    Output: Returns no object.
    Prints the following:
    1) Number of recipes / reviews that satisfy the condition
    2) Total number of recipes / reviews in the dataframe
    3) Percentage of recipes / reviews that satisfy the condition
    """
    # Find out num rows satisfying the condition
    if aggregation_level == "recipe": 
        number_of_rows_satisfying_condition = (df
                                             .filter(condition)
                                             .agg(F.countDistinct("id"))).first()[0]
      
        n_rows_total = (df.agg(F.countDistinct("id"))).first()[0]
    if aggregation_level == "review":
        number_of_rows_satisfying_condition = (df
                                             .filter(condition)
                                             .agg(F.countDistinct("id","user_id"))).first()[0]
        n_rows_total = (df.agg(F.countDistinct("id","user_id"))).first()[0]
  
    # Find out % rows satisfying the conditon and print a properly formatted output
    perc_rows = round(number_of_rows_satisfying_condition * 100/ n_rows_total, 2)
    print('Condition String                   : "{}"'.format(condition))
    print("Num {}s Satisfying Condition   : {} [{}%]".format(aggregation_level.title(), number_of_rows_satisfying_condition, perc_rows))
    print("Total Num {}s                  : {}".format(aggregation_level.title(), n_rows_total))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the data

- Read ```interaction_level_df_processed```

In [12]:
interaction_level_df = spark.read.parquet(
                                          # add code here
                                          "s3://sparkml/receiperecommender/interaction_level_df.parquet/"
                                          )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert interaction_level_df.count() == 1132367, "There is a mistake in reading the data."
assert len(interaction_level_df.columns) == 33, "There is a mistake in reading the data."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
interaction_level_df.show(5)

## Bucketing and Cleaning Numerical Features

#### **1. `years_since_submission_on_review_date`** 
[Review Time Since Submission]
- Recipes more than 6 years old are rated low

In [14]:
get_quantiles(df = interaction_level_df,
              col_name = "years_since_submission_on_review_date")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: -4, 0.01: -4.833333333333333, 0.25: 0.7177419358333333, 0.5: 2.7123655916666665, 0.75: 4.4247311824999995, 0.99: 19.284946236666666, 1.0: 19}

In [15]:
get_n_items_satisfying_condition(df = interaction_level_df,
                                 condition= 'years_since_submission_on_review_date < 0',
                                 aggregation_level= "review")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Condition String                   : "years_since_submission_on_review_date < 0"
Num Reviews Satisfying Condition   : 1246 [0.11%]
Total Num Reviews                  : 1132367

In [16]:
# Only keep interactions with review dates >= recipe submission date

interaction_level_df = (interaction_level_df
                        .filter('years_since_submission_on_review_date >= 0'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
splits = [ 0, 1, 3, 6, float('Inf')]
inputCol  = "years_since_submission_on_review_date"
outputCol = "years_since_submission_on_review_date_bucket"

(interaction_level_df, submission_time_bucketizer, submission_time_pandas_df) = bucket_col_print_summary(df = interaction_level_df,
                                                                              splits = splits,
                                                                              inputCol  = inputCol,
                                                                              outputCol = outputCol)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Added bucketized column years_since_submission_on_review_date_bucket

Bucketing done for split definition: [0, 1, 3, 6, inf]

Printing summary statistics for ratings in buckets below:

In [ ]:
interaction_level_df.columns

In [ ]:
submission_time_bucketizer.columns

In [ ]:
submission_time_bucketizer.show(5)

In [ ]:
%matplot plt

#### **2. `minutes`** 

[prep time]
- Somewhat relevant
- Low prep time is preferred

In [18]:
get_quantiles(df = submission_time_bucketizer,
              col_name = "minutes")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 0, 0.01: 0.0, 0.25: 20.0, 0.5: 35.0, 0.75: 55.0, 0.99: 2147483647.0, 1.0: 2147483647}

In [19]:
# Capping prep time at 930 minutes
get_n_items_satisfying_condition(df = submission_time_bucketizer,
                                 condition= 'minutes <= 930',
                                 aggregation_level= "recipe")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Condition String                   : "minutes <= 930"
Num Recipes Satisfying Condition   : 229205 [99.01%]
Total Num Recipes                  : 231501

In [20]:
# Only keep interactions with  prep time less than equal to 930 minutes

interaction_level_df = (submission_time_bucketizer
                        .filter('minutes <= 930'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
submission_time_bucketizer.select('minutes', 'rating','n_steps').show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-------+
|minutes|rating|n_steps|
+-------+------+-------+
|265    |4     |22     |
|35     |5     |7      |
|35     |5     |7      |
|35     |4     |7      |
|35     |4     |7      |
+-------+------+-------+
only showing top 5 rows

In [22]:
# investigating recipes with minutes = 0 -> Look at n_steps for such recipes.

get_column_distribution_summary(
# add code here 
    submission_time_bucketizer.filter('minutes=0'), ['n_steps','rating','recipe_id']
)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+---------+---------+
|n_steps|        avg_rating|     stddev_rating|n_ratings|n_recipes|
+-------+------------------+------------------+---------+---------+
|      1|              4.24|1.0908712114635715|       25|       12|
|      2|4.4423076923076925|  1.08678668673504|      104|       28|
|      3| 3.989130434782609|1.5750414356065066|      184|       44|
|      4|  4.30635838150289|1.3867374413641127|      173|       57|
|      5| 4.231788079470198| 1.356386192466306|      302|       90|
|      6|       4.470703125| 1.146389334652367|      512|      102|
|      7|4.3447432762836184|   1.2875641979464|      409|       92|
|      8| 4.381995133819951|1.2774085466671241|      411|       92|
|      9| 4.076190476190476|1.5270088873176948|      315|       86|
|     10| 4.303398058252427|1.3500249999653713|      412|       77|
|     11|4.3226744186046515|1.3216995194200645|      344|       60|
|     12| 4.258536585365854|1.3308142151970859| 

In [23]:
# let's look at some examples with 1 step only to see if this makes sense
submission_time_bucketizer.filter('n_steps=1').show(5, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+---------------+---+-------+--------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------

In [24]:
submission_time_bucketizer.select('minutes', 'rating','n_steps').show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-------+
|minutes|rating|n_steps|
+-------+------+-------+
|265    |4     |22     |
|35     |5     |7      |
|35     |5     |7      |
|35     |4     |7      |
|35     |4     |7      |
+-------+------+-------+
only showing top 5 rows

In [25]:
get_n_items_satisfying_condition(df = submission_time_bucketizer,
                                 condition= 'minutes == 0',
                                 aggregation_level= "recipe")
#
#'''
#Expected Results : 
#Condition String                   : "minutes == 0"
#Num Recipes Satisfying Condition   : 1094 [0.47%]
#Total Num Recipes                  : 231501

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Condition String                   : "minutes == 0"
Num Recipes Satisfying Condition   : 1094 [0.47%]
Total Num Recipes                  : 231501

In [ ]:
submission_time_bucketizer.select('minutes', 'rating','n_steps').show(5, truncate=False)

In [26]:
# Remove recipes with cook time zero
interaction_level_df = (submission_time_bucketizer
                        .filter('minutes > 0'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
interaction_level_df.select('minutes', 'rating','n_steps').show(5, truncate=False)

In [27]:
inputCol  = "minutes"
outputCol = "prep_time_bucket"
splits = [ 0, 5, 15, 30, 60, 300, 900, float('Inf')]
bucket = Bucketizer(splits=splits, inputCol=inputCol, outputCol=outputCol)
bucketizer = bucket.transform(interaction_level_df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
bucketizer.groupby(outputCol).count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------+
|prep_time_bucket| count|
+----------------+------+
|             1.0|122052|
|             6.0| 11715|
|             5.0| 46566|
|             2.0|237323|
|             4.0|312083|
|             0.0| 17981|
|             3.0|378891|
+----------------+------+

In [ ]:
bucketizer.groupby(outputCol).mean('rating').show()

In [ ]:
interaction_level_df.columns

In [29]:
splits = [ 0, 5, 15, 30, 60, 300, 900, float('Inf')]
inputCol  = "minutes"
outputCol = "prep_time_bucket"

(interaction_level_df, prep_time_bucketizer, submission_time_pandas_df) = bucket_col_print_summary(df = interaction_level_df,
                                                                              splits = splits,
                                                                              inputCol  = inputCol,
                                                                              outputCol = outputCol)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Added bucketized column prep_time_bucket

Bucketing done for split definition: [0, 5, 15, 30, 60, 300, 900, inf]

Printing summary statistics for ratings in buckets below:

In [ ]:
prep_time_bucketizer.columns

In [ ]:
# this will display the plot 
%matplot plt

**3. `n_steps`**

- Clearly relevant
- Recipes with less than 2 steps are rated high
- Recipes with more than 29 steps are rated very low

In [30]:
get_quantiles(df = prep_time_bucketizer,
              col_name = "n_steps")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 0, 0.01: 0.0, 0.25: 5.0, 0.5: 8.0, 0.75: 10.0, 0.99: 145.0, 1.0: 145}

In [31]:
get_n_items_satisfying_condition(df = prep_time_bucketizer,
                                 condition= 'n_steps < 0',
                                 aggregation_level= "review")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Condition String                   : "n_steps < 0"
Num Reviews Satisfying Condition   : 0 [0.0%]
Total Num Reviews                  : 1126611

In [32]:
# Remove recipes with n_steps zero

interaction_level_df = (prep_time_bucketizer
                        .filter('n_steps > 0'))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
interaction_level_df.count()

In [ ]:
interaction_level_df.select('n_steps').distinct().collect()

In [33]:
splits = [ 0, 2, 4, 6, 8, 10, float('Inf')]
inputCol  = "n_steps"
outputCol = "n_steps_bucket"

(interaction_level_df, n_steps_bucketizer, submission_time_pandas_df) = bucket_col_print_summary(df = interaction_level_df,
                                                                              splits = splits,
                                                                              inputCol  = inputCol,
                                                                              outputCol = outputCol)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Added bucketized column n_steps_bucket

Bucketing done for split definition: [0, 2, 4, 6, 8, 10, inf]

Printing summary statistics for ratings in buckets below:

In [ ]:
# this will display the plot 
%matplot plt

**4. `n_ingredients`**
- Not relevant

In [34]:
get_quantiles(df = n_steps_bucketizer,
              col_name = "n_ingredients")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 1, 0.01: 1.0, 0.25: 6.0, 0.5: 8.0, 0.75: 10.0, 0.99: 43.0, 1.0: 43}

In [35]:
get_n_items_satisfying_condition(df = n_steps_bucketizer,
                                 condition= 'n_ingredients >= 0',
                                 aggregation_level= "review")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Condition String                   : "n_ingredients >= 0"
Num Reviews Satisfying Condition   : 1126609 [100.0%]
Total Num Reviews                  : 1126609

In [36]:
interaction_level_df = n_steps_bucketizer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
splits = [ 0, 5, 10, 20, float('Inf')]
inputCol  = "n_ingredients"
outputCol = "n_ingredients_bucket"

(interaction_level_df, n_ingredients_bucketizer, submission_time_pandas_df) = bucket_col_print_summary(df = interaction_level_df,
                                                                              splits = splits,
                                                                              inputCol  = inputCol,
                                                                              outputCol = outputCol)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Added bucketized column n_ingredients_bucket

Bucketing done for split definition: [0, 5, 10, 20, inf]

Printing summary statistics for ratings in buckets below:

In [ ]:
# this will display the plot 
%matplot plt

**5. `nutrition` columns**

- `calories` - Calories per serving seems irrelevant
- `fat (per 100 cal)` - Calories per serving seems irrelevant
- `sugar (per 100 cal)` - Calories per serving seems irrelevant
- `sodium (per 100 cal)` - Calories per serving seems irrelevant
- `protein (per 100 cal)` - Calories per serving seems irrelevant
- `sat. fat (per 100 cal)` - Calories per serving seems irrelevant
- `carbs (per 100 cal)` - Calories per serving seems irrelevant

In [38]:
interaction_level_df = n_ingredients_bucketizer


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
interaction_level_df.select('minutes', 'rating','n_steps').show(5, truncate=False)

In [ ]:
interaction_level_df.columns

In [39]:
nutrition_cols = ['calories', 
                  'total_fat_PDV', 
                  'sugar_PDV', 
                  'sodium_PDV', 
                  'protein_PDV', 
                  'saturated_fat_PDV', 
                  'carbohydrates_PDV', 
                  'total_fat_per_100_cal', 
                  'sugar_per_100_cal', 
                  'sodium_per_100_cal', 
                  'protein_per_100_cal', 
                  'saturated_fat_per_100_cal', 
                  'carbohydrates_per_100_cal']



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import col,isnan,when,count
#check if there is any null values in the nutrition columns
interaction_level_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in nutrition_cols]
   ).show()

In [40]:
# Write code to find the quantile distribution of all the above columns
nutrition_cols_quantile_dict = {}
for col in nutrition_cols:
    # add code here 
    quant_dict = get_quantiles(df = interaction_level_df, col_name = col,quantiles_list = [0.25, 0.5, 0.75, 0.95])
    lst_quantile = []
    for quant in quant_dict.values():
        if(int(quant) not in lst_quantile):
            lst_quantile.append(int(quant))
    lst_quantile.append(float('Inf'))
    nutrition_cols_quantile_dict[col]=lst_quantile
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
nutrition_cols_quantile_dict

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'calories': [0, 163, 365, 377, 434360, inf], 'total_fat_PDV': [0, 7, 26, 28, 17183, inf], 'sugar_PDV': [0, 7, 18, 39, 362729, inf], 'sodium_PDV': [0, 5, 21, 24, 29338, inf], 'protein_PDV': [0, 6, 13, 30, 6552, inf], 'saturated_fat_PDV': [0, 6, 31, 35, 10395, inf], 'carbohydrates_PDV': [0, 3, 7, 12, 36098, inf], 'total_fat_per_100_cal': [0, 3, 6, 7, 18, inf], 'sugar_per_100_cal': [0, 2, 6, 18, 108, inf], 'sodium_per_100_cal': [0, 1, 5, 6, 199909, inf], 'protein_per_100_cal': [0, 2, 7, 8, 50, inf], 'saturated_fat_per_100_cal': [0, 3, 6, 10, 50, inf], 'carbohydrates_per_100_cal': [0, 1, 3, 4, 16, inf]}

In [ ]:
nutrition_cols_quantile_dict['calories']

In [41]:
#nutrition_col_quantile_summary = # set index for the dataframe 
for col in nutrition_cols:
    
    # programatically diclare the arguments of Bucketizer
    #print('col',col)
    splits = nutrition_cols_quantile_dict[col]
    inputCol  = col
    outputCol = col+'_bucket'
    
    # drop column if it already exsists 
    if outputCol in interaction_level_df.columns:
        interaction_level_df = interaction_level_df.drop(outputCol)
        bucketizer_df = bucketizer_df.drop(outputCol)



    # Training bucketizer
    bucket = Bucketizer(splits = splits,inputCol  = inputCol,outputCol = outputCol)
   
    # apply the bucketizer
    bucketizer_df = bucket.transform(interaction_level_df)
    
    # create the summary df   
    # the output in the cells below is in the form of pandas but you can keep your result in pyspark as well. 
    interaction_level_df = bucketizer_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# set the max columns to none
pd.set_option('display.max_columns', None)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#nutrition_col_quantile_summary

In [43]:
## Writing the modified data to S3 

interaction_level_df.write.mode('overwrite').parquet(
                                                        "s3://sparkml/EDA02/interaction_level_df.parquet"
                                                    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…